In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [2]:
# !pip install loompy

In [3]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [4]:

from sklearn.model_selection import train_test_split

In [5]:
import scanpy as sc

In [6]:
# pip install loompy

In [7]:
adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [8]:
adata_sc

AnnData object with n_obs × n_vars = 3005 × 19972
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify'

In [9]:
adata_st

AnnData object with n_obs × n_vars = 4530 × 33
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify'

In [10]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [11]:
info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

In [12]:
adata_sc = adata_sc[:,info_gene]
adata_st = adata_st[:,info_gene]

In [13]:
import random 
random.seed(2023)
g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
g1 = sorted(g1)

In [14]:
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

In [15]:
test_g

['Itpr2', 'Hexb', 'Pthlh', 'Plp1', 'Mrc1', 'Crh', 'Gfap']

In [16]:
# sc.pp.filter_genes(adata_sc, min_counts=1)

In [17]:
pv = int(len(test_g)/2)
if pv > 100:
    pv = 100

In [18]:
adata_st = adata_st[:, train_g]

In [19]:
osmFISH_data = adata_st.to_df()
RNA_data = adata_sc.to_df()

In [20]:
np.sum(np.isnan(RNA_data.T.values)*1)

0

In [21]:
# adata_st_old = adata_st.copy()

In [22]:
# for prop in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
#     lightning.seed_everything(0)

#     adata_st = adata_st_old.copy()
#     sc.pp.subsample(adata_st, fraction = prop, random_state = 0)
    
#     osmFISH_data = adata_st.to_df()

#     Gene_set = test_g
#     Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
#                                genes_to_predict = Gene_set)
#     Imp_Genes.to_csv(f"spage_imputation_smfishexample_updategene_prop{prop}.csv")

In [25]:
# adata_st_old = adata_st.copy()
scrna_data_old = adata_sc.copy()

In [26]:
for prop in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    lightning.seed_everything(0)

    adata_sc = scrna_data_old.copy()
    sc.pp.subsample(adata_sc, fraction = prop, random_state = 0)
    
    osmFISH_data = adata_st.to_df()
    RNA_data = adata_sc.to_df()

    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"spage_imputation_smfishexample_updategene_scprop{prop}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0


In [4]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [5]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [6]:

from sklearn.model_selection import train_test_split

In [7]:
import scanpy as sc

In [8]:
# adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
# adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# # adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# # adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [9]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [ ]:
adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
adata_sc.obs['scClassify'] = adata_sc.obs['graph_cluster_anno'].copy()
adata_sc.var_names_make_unique()

In [ ]:
for seed in range(0,10):
    lightning.seed_everything(0)

    adata_st = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/human_breast_simulation/spe_xenium_data_0.1_seed{seed}.h5ad")
    info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

    adata_sc = adata_sc[:,info_gene]
    adata_st = adata_st[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
    g1 = sorted(g1)

    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    pv = int(len(test_g)/2)
    if pv > 100:
        pv = 100

    adata_st = adata_st[:, train_g]

    osmFISH_data = adata_st.to_df()
    RNA_data = adata_sc.to_df()
    
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"/gpfs/gibbs/pi/zhao/tl688/tangram/SpaGE/spage_imputation_humanbreast_updategene_seed{seed}.csv")

In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [2]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [3]:

from sklearn.model_selection import train_test_split

In [4]:
import scanpy as sc

In [5]:
# adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
# adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# # adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# # adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [6]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = sorted(list(set(adata_sc.var_names).intersection(adata_st.var_names)))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [7]:
for seed in range(0,10):
    lightning.seed_everything(0)

    adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
    adata_sc.obs['scClassify'] = adata_sc.obs['cell_type_alias_label2'].copy()
    adata_sc.var_names_make_unique()

    adata_st = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/spe_xenium_data_0.1_seed{seed}.h5ad")
    info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

    adata_sc = adata_sc[:,info_gene]
    adata_st = adata_st[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
    g1 = sorted(g1)

    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    pv = int(len(test_g)/2)
    if pv > 100:
        pv = 100

    adata_st = adata_st[:, train_g]

    osmFISH_data = adata_st.to_df()
    RNA_data = adata_sc.to_df()
    
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"/gpfs/gibbs/pi/zhao/tl688/tangram/SpaGE/spage_imputation_brain_updategene_seed{seed}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0


In [11]:
adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/starmap/scrnaseq.h5ad")
adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/starmap/spatialseq.h5ad")

# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/starmap/scrnaseq.h5ad")
# spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/starmap/spatialseq.h5ad")

# adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [12]:
adata_sc

AnnData object with n_obs × n_vars = 15413 × 45768
    obs: 'sample_name', 'sample_id', 'sample_type', 'organism', 'donor', 'sex', 'age_days', 'eye_condition', 'genotype', 'driver_lines', 'reporter_lines', 'brain_hemisphere', 'brain_region', 'brain_subregion', 'injection_label_direction', 'injection_primary', 'injection_secondary', 'injection_tract', 'injection_material', 'injection_exclusion_criterion', 'facs_date', 'facs_container', 'facs_sort_criteria', 'rna_amplification_set', 'library_prep_set', 'library_prep_avg_size_bp', 'seq_name', 'seq_tube', 'seq_batch', 'total_reads', 'percent_exon_reads', 'percent_intron_reads', 'percent_intergenic_reads', 'percent_rrna_reads', 'percent_mt_exon_reads', 'percent_reads_unique', 'percent_synth_reads', 'percent_ecoli_reads', 'percent_aligned_reads_total', 'complexity_cg', 'genes_detected_cpm_criterion', 'genes_detected_fpkm_criterion', 'tdt_cpm', 'gfp_cpm', 'class', 'subclass', 'cluster', 'confusion_score', 'cluster_correlation', 'core_intermed

In [13]:
adata_st

AnnData object with n_obs × n_vars = 1207 × 53
    obs: 'scClassify', 'ori_label'
    obsm: 'spatial'

In [14]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [15]:
info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

In [16]:
adata_sc = adata_sc[:,info_gene]
adata_st = adata_st[:,info_gene]

In [17]:
import random 
random.seed(2023)
g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
g1 = sorted(g1)

In [18]:
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

In [19]:
test_g

['Pthlh',
 'Cldn5',
 'Cmtm5',
 'Ctso',
 'Slc20a1',
 '2810468N07Rik',
 'Mt2',
 'Ddah1',
 'Aqp4',
 'Tmem91',
 'Rorb',
 'Reln',
 'Tusc3',
 'Pou3f4',
 'Acsbg1',
 'Bsg',
 'Ptgds',
 'Ntsr2']

In [20]:
# sc.pp.filter_genes(adata_sc, min_counts=1)

In [21]:
pv = int(len(test_g)/2)
if pv > 100:
    pv = 100

In [22]:
adata_st = adata_st[:, train_g]

In [23]:
osmFISH_data = adata_st.to_df()
RNA_data = adata_sc.to_df()

In [24]:
np.sum(np.isnan(RNA_data.T.values)*1)

0

In [25]:
for i in range(0,10):
    lightning.seed_everything(i)
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"spage_imputation_starmap_updategene_seed{i}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 1
[rank: 0] Global seed set to 2
[rank: 0] Global seed set to 3
[rank: 0] Global seed set to 4
[rank: 0] Global seed set to 5
[rank: 0] Global seed set to 6
[rank: 0] Global seed set to 7
[rank: 0] Global seed set to 8
[rank: 0] Global seed set to 9


# Normalized

In [31]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [32]:
# !pip install loompy

In [33]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [34]:

from sklearn.model_selection import train_test_split

In [35]:
import scanpy as sc

In [36]:
# pip install loompy

In [37]:
adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [38]:
adata_sc

AnnData object with n_obs × n_vars = 3005 × 19972
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify'

In [39]:
adata_st

AnnData object with n_obs × n_vars = 4530 × 33
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify'

In [40]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [41]:
info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

In [42]:
adata_sc = adata_sc[:,info_gene]
adata_st = adata_st[:,info_gene]

In [43]:
import random 
random.seed(2023)
g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
g1 = sorted(g1)

In [44]:
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

In [45]:
test_g

['Itpr2', 'Hexb', 'Pthlh', 'Plp1', 'Mrc1', 'Crh', 'Gfap']

In [46]:
# sc.pp.filter_genes(adata_sc, min_counts=1)

In [47]:
pv = int(len(test_g)/2)
if pv > 100:
    pv = 100

In [48]:
adata_st = adata_st[:, train_g]

In [49]:
osmFISH_data = adata_st.to_df()
RNA_data = adata_sc.to_df()

In [50]:
# Normalization
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
RNA_data = RNA_data.apply(Log_Norm_cpm,axis=0)

In [51]:
# Normalization
cell_count = np.sum(osmFISH_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)
osmFISH_data = osmFISH_data.apply(Log_Norm_spatial,axis=0)

In [52]:
for i in range(0,10):
    lightning.seed_everything(i)
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"spage_imputation_smfishexample_normalized_seed{i}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 1
[rank: 0] Global seed set to 2
[rank: 0] Global seed set to 3
[rank: 0] Global seed set to 4
[rank: 0] Global seed set to 5
[rank: 0] Global seed set to 6
[rank: 0] Global seed set to 7
[rank: 0] Global seed set to 8
[rank: 0] Global seed set to 9


In [6]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [7]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [8]:

from sklearn.model_selection import train_test_split

In [9]:
import scanpy as sc

In [10]:
# adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
# adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# # adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# # adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [11]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = sorted(list(set(adata_sc.var_names).intersection(adata_st.var_names)))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [14]:
for seed in range(0,10):
    lightning.seed_everything(0)

    adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/scRNAseq/seqfish/scRNAseq_seqfish.h5ad")
    adata_st = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/seqfish_data_seed{seed}.h5ad")
    adata_sc.var_names_make_unique()
    
    adata_sc.obs['scClassify'] = adata_sc.obs['celltype'].copy() 
    adata_st.obs['scClassify'] = adata_st.obs['celltype_mapped_refined'].copy() 
    info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

    adata_sc = adata_sc[:,info_gene]
    adata_st = adata_st[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
    g1 = sorted(g1)

    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    pv = int(len(test_g)/2)
    if pv > 100:
        pv = 100

    adata_st = adata_st[:, train_g]

    osmFISH_data = adata_st.to_df()
    RNA_data = adata_sc.to_df()
    
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"/gpfs/gibbs/pi/zhao/tl688/tangram/SpaGE/spage_imputation_seqfish_updategene_seed{seed}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0


In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
from SpaGE.main import SpaGE
import warnings
import lightning
warnings.filterwarnings('ignore')

In [2]:
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

In [3]:

from sklearn.model_selection import train_test_split

In [4]:
import scanpy as sc

In [5]:
# adata_sc = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
# adata_st = sc.read("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

# # adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
# # adata_st = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")

In [6]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = sorted(list(set(adata_sc.var_names).intersection(adata_st.var_names)))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [7]:
for seed in range(0,10):
    lightning.seed_everything(0)

    adata_sc = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/lungcancer/scrnaseq_lungcancer.h5ad")
    adata_st = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_lungcancer/spatial_lungcancer_data_seed{seed}.h5ad")
    adata_sc.var_names_make_unique()
    
    adata_sc.obs['scClassify'] = adata_sc.obs['Cell_type'].copy() 
    adata_st.obs['scClassify'] = adata_st.obs['Class'].copy() 
    info_gene = calcualte_pse_correlation(adata_sc, adata_st, 'scClassify')

    adata_sc = adata_sc[:,info_gene]
    adata_st = adata_st[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(adata_st.var_names).intersection(adata_sc.var_names))
    g1 = sorted(g1)

    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    pv = int(len(test_g)/2)
    if pv > 100:
        pv = 100

    adata_st = adata_st[:, train_g]

    osmFISH_data = adata_st.to_df()
    RNA_data = adata_sc.to_df()
    
    Gene_set = test_g
    Imp_Genes = SpaGE(osmFISH_data,RNA_data,n_pv=pv,
                               genes_to_predict = Gene_set)
    Imp_Genes.to_csv(f"/gpfs/gibbs/pi/zhao/tl688/tangram/SpaGE/spage_imputation_lungcancer_updategene_seed{seed}.csv")

[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
[rank: 0] Global seed set to 0
